### DK Salaries Optimizer

In [1]:
import pandas as pd
import numpy as np
from pulp import *

In [16]:
dk_data = pd.read_csv("dksalaries_nba.csv")

# Get the fields that are useful
dk_data = dk_data[['Name', 'Roster Position', 'Salary', 'TeamAbbrev', 'AvgPointsPerGame']]

dk_data.head()

,Name,Roster Position,Salary,TeamAbbrev,AvgPointsPerGame
0,Giannis Antetokounmpo,PF/F/UTIL,11400,MIL,59.39
1,James Harden,PG/SG/G/UTIL,11200,HOU,59.62
2,Russell Westbrook,PG/G/UTIL,8900,HOU,50.06
3,Jimmy Butler,SG/SF/F/G/UTIL,8300,MIA,41.53
4,Chris Paul,PG/G/UTIL,8200,OKC,37.53


In [17]:
injured_players = ['Gordon Hayward', 'Eric Bledsoe']

In [18]:
dk_data = dk_data[~dk_data['Name'].isin(injured_players)]

In [19]:
# Roster Positions for NBA 
nba_roster_positions = ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UTIL']

SALARY_CAP = 50000
MAX_PLAYERS = 8


def make_vars(name, position, roster_position):
    if position in roster_position:
        return LpVariable(name + '_' + position,lowBound=0, cat='Binary')
    return 0

for position in nba_roster_positions:
    dk_data[position] = dk_data.apply(lambda x: make_vars(x['Name'],position, x['Roster Position']),axis=1)

    #[1 if position in x else 0 for x in dk_data['Roster Position']]

In [20]:
prob = LpProblem("OptimizeDK",LpMaximize)

In [21]:
names = dk_data['Name']
salaries = dk_data['Salary']
projected_points = dk_data['AvgPointsPerGame']

In [22]:
dk_data.head()

,Name,Roster Position,Salary,TeamAbbrev,AvgPointsPerGame,PG,SG,SF,PF,C,G,F,UTIL
0,Giannis Antetokounmpo,PF/F/UTIL,11400,MIL,59.39,0,0,0,Giannis_Antetokounmpo_PF,0,0,Giannis_Antetokounmpo_F,Giannis_Antetokounmpo_UTIL
1,James Harden,PG/SG/G/UTIL,11200,HOU,59.62,James_Harden_PG,James_Harden_SG,0,0,0,James_Harden_G,0,James_Harden_UTIL
2,Russell Westbrook,PG/G/UTIL,8900,HOU,50.06,Russell_Westbrook_PG,0,0,0,0,Russell_Westbrook_G,0,Russell_Westbrook_UTIL
3,Jimmy Butler,SG/SF/F/G/UTIL,8300,MIA,41.53,0,Jimmy_Butler_SG,Jimmy_Butler_SF,0,0,Jimmy_Butler_G,Jimmy_Butler_F,Jimmy_Butler_UTIL
4,Chris Paul,PG/G/UTIL,8200,OKC,37.53,Chris_Paul_PG,0,0,0,0,Chris_Paul_G,0,Chris_Paul_UTIL


In [23]:
# The objective function is added to 'prob' first
# OBJ FUNCTION
prob += lpSum([dk_data[i] * projected_points for i in nba_roster_positions]), 'Projected Points'

In [24]:
# CONSTRAINTS

# Stay under salary cap
prob += lpSum(salaries * dk_data[i] for i in nba_roster_positions) <= SALARY_CAP

# Stay under Num Players
prob += lpSum(dk_data[i] for i in nba_roster_positions) <= MAX_PLAYERS

# Select one of each position
for position in nba_roster_positions:
    prob += lpSum([dk_data[position]]) >= 1

# Don't use same player twice
for index, row in dk_data.iterrows():
    prob += lpSum(row[i] for i in nba_roster_positions) <= 1


In [25]:
# prob

In [26]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [27]:
prob.status

1

In [28]:
print("Total Estimated Points = ", value(prob.objective))


Total Estimated Points =  277.01


In [29]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    if v.varValue == 1:
        print(v.name)
        v.getName

Derrick_Jones_Jr._SF
Donte_DiVincenzo_G
Giannis_Antetokounmpo_PF
Jae_Crowder_F
James_Harden_UTIL
Kendrick_Nunn_SG
Nerlens_Noel_C
Russell_Westbrook_PG
